In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox
import numpy as np

# Global variable for PGM
pgm = None

def load_text():
    file_name = filedialog.askopenfilename(title="Select file", filetypes=(("Text files", "*.txt"),))
    if file_name:
        try:
            with open(file_name, 'r') as file:
                text = file.read()
            text_input.delete("1.0", tk.END)
            text_input.insert("1.0", text)
        except Exception as e:
            messagebox.showerror("Error", f"Failed to read file: {str(e)}")

def encode():
    msg = text_input.get("1.0", "end-1c").replace("\n", "").strip()
    if not all(c in '01' for c in msg):
        messagebox.showerror("Error", "Input is not in binary format.")
        return
    encoded_msg = hamming_encoder(msg)
    encoded_output.delete("1.0", tk.END)
    encoded_output.insert("1.0", encoded_msg)

import numpy as np
import tkinter as tk
from tkinter import messagebox

import numpy as np
import tkinter as tk
from tkinter import messagebox

def generate_pgm(n, k, d):
    if n <= k or d > n - k + 1:
        messagebox.showerror("Error", f"Invalid parameters: Ensure n > k and d ≤ n-k+1. Given: n={n}, k={k}, d={d}, n-k+1={n-k+1}")
        return None

    # Initialize PGM with zeros
    pgm = np.zeros((k, n), dtype=int)

    # Fill the first k columns with an identity matrix for data encoding
    np.fill_diagonal(pgm[:, :k], 1)

    # Dynamic generation of parity bits
    # Skip powers of two and map other indices dynamically into the last 'd' columns
    x = 0
    index = k  # Start placing parity bits right after the last data bit column
    for i in range(1, n+1 ):
        if i == 2 ** x:
            x += 1  # Skip powers of two
        else:
            if index < n:  # Ensure we do not exceed the matrix width
                # Generate parity bits based on a simple pattern or a complex function as needed
                binary_pattern = [int(b) for b in f'{i:0{d}b}'[-d:]]  # Convert index to binary, take last 'd' bits
                for j in range(d):
                    if index + j < n:  # Check within bounds for columns
                        pgm[i-1-x, index + j] = binary_pattern[j]
                

    return pgm

def hamming_encoder(binary_input):
    if pgm is None:
        messagebox.showerror("Error", "PGM is not generated. Please generate before encoding.")
        return ""
    k = pgm.shape[0]
    binary_blocks = [binary_input[i:i+k] for i in range(0, len(binary_input), k)]
    encoded_blocks = []
    for block in binary_blocks:
        if len(block) < k:
            block += '0' * (k - len(block))
        block_vector = np.array(list(map(int, block)))
        encoded_vector = np.dot(block_vector, pgm) % 2
        encoded_blocks.append(''.join(map(str, encoded_vector)))
    return ''.join(encoded_blocks)


def permute_rows():
    global pgm
    try:
        a = int(a_entry.get()) - 1  # Convert to zero-indexed
        b = int(b_entry.get()) - 1  # Convert to zero-indexed
        if not (0 <= a < pgm.shape[0]) or not (0 <= b < pgm.shape[0]):
            messagebox.showerror("Error", "Row indices are out of range. Please enter valid row numbers.")
            return
        pgm[[a, b]] = pgm[[b, a]]
        update_pgm_display()
        messagebox.showinfo("Success", f"Rows {a+1} and {b+1} successfully swapped.")
    except ValueError:
        messagebox.showerror("Error", "Invalid input. Please enter integer values for row indices.")


def restore_pgm():
    global pgm
    if pgm_original is not None:
        pgm = np.copy(pgm_original)
        update_pgm_display()

def update_pgm_display():
    if pgm is None:
        pgm_display.insert("1.0", "No PGM generated yet.")
        return
    pgm_display.delete("1.0", tk.END)
    pgm_str = '\n'.join(' '.join(str(cell) for cell in row) for row in pgm)
    pgm_display.insert("1.0", pgm_str)

def generate_pgm_gui():
    global pgm, pgm_original
    try:
        n = int(n_entry.get())
        k = int(k_entry.get())
        d = int(d_entry.get())
        pgm = generate_pgm(n, k, d)
        if pgm is not None:
            pgm_original = np.copy(pgm)  # Store original for restoration
            update_pgm_display()
    except ValueError:
        messagebox.showerror("Error", "Invalid input. Please enter integer values.")

root = tk.Tk()
root.title("Hamming Encoder GUI")

param_frame = tk.Frame(root)
param_frame.pack(pady=10)
n_label = tk.Label(param_frame, text="n:")
n_label.pack(side=tk.LEFT)
n_entry = tk.Entry(param_frame, width=5)
n_entry.pack(side=tk.LEFT, padx=5)

k_label = tk.Label(param_frame, text="k:")
k_label.pack(side=tk.LEFT)
k_entry = tk.Entry(param_frame, width=5)
k_entry.pack(side=tk.LEFT, padx=5)

d_label = tk.Label(param_frame, text="d:")
d_label.pack(side=tk.LEFT)
d_entry = tk.Entry(param_frame, width=5)
d_entry.pack(side=tk.LEFT, padx=5)

generate_button = tk.Button(param_frame, text="Generate PGM", command=generate_pgm_gui)
generate_button.pack(side=tk.LEFT, padx=10)

# Text input
text_input_frame = tk.Frame(root)
text_input_frame.pack(pady=10)
text_input = tk.Text(text_input_frame, height=5, width=50)
text_input.pack(side=tk.LEFT)

# Buttons for text input operations
button_frame = tk.Frame(text_input_frame)
button_frame.pack(side=tk.LEFT, padx=5)
load_button = tk.Button(button_frame, text="Load Text File", command=load_text)
load_button.pack(fill=tk.X)
encode_button = tk.Button(button_frame, text="Encode", command=encode)
encode_button.pack(fill=tk.X)

# Entry widgets for row indices
index_frame = tk.Frame(root)
index_frame.pack(pady=10)
a_label = tk.Label(index_frame, text="Row a:")
a_label.pack(side=tk.LEFT)
a_entry = tk.Entry(index_frame, width=5)
a_entry.pack(side=tk.LEFT, padx=5)
b_label = tk.Label(index_frame, text="Row b:")
b_label.pack(side=tk.LEFT)
b_entry = tk.Entry(index_frame, width=5)
b_entry.pack(side=tk.LEFT, padx=5)

# Buttons for matrix operations
matrix_button_frame = tk.Frame(root)
matrix_button_frame.pack(pady=10)
permute_button = tk.Button(matrix_button_frame, text="Permute Rows", command=permute_rows)
permute_button.pack(side=tk.LEFT, padx=5)
restore_button = tk.Button(matrix_button_frame, text="Restore PGM", command=restore_pgm)
restore_button.pack(side=tk.LEFT, padx=5)

# PGM display
pgm_display = tk.Text(root, height=11, width=75, bg="dark grey")
pgm_display.pack(pady=10)
update_pgm_display()

# Encoded output
encoded_output = tk.Text(root, height=5, width=50, bg="dark cyan")
encoded_output.pack(pady=10)

root.mainloop()



In [ ]:
# import tkinter as tk
# from tkinter import filedialog, messagebox
# import numpy as np

# # Define the Parity Generation Matrix (PGM)
# pgm_original = np.array([
#     [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
#     [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0],
#     [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
#     [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0],
#     [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1],
#     [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1],
#     [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1],
#     [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1],
#     [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1],
#     [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1],
#     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
    
# ], dtype=int)
# pgm = np.copy(pgm_original)

# def load_text():
#     file_name = filedialog.askopenfilename(title="Select file", filetypes=(("Text files", "*.txt"),))
#     if file_name:
#         try:
#             with open(file_name, 'r') as file:
#                 text = file.read()
#             text_input.delete("1.0", tk.END)
#             text_input.insert("1.0", text)
#         except Exception as e:
#             messagebox.showerror("Error", f"Failed to read file: {str(e)}")

# def encode():
#     msg = text_input.get("1.0", "end-1c").replace("\n", "")
#     encoded_msg = hamming_encoder(msg)
#     encoded_output.delete("1.0", tk.END)
#     encoded_output.insert("1.0", encoded_msg)

# def hamming_encoder(msg):
#     binary_input = ''.join(format(ord(x), '08b') for x in msg)
#     binary_blocks = [binary_input[i:i+11] for i in range(0, len(binary_input), 11)]
    
#     encoded_blocks = []
#     for block in binary_blocks:
#         if len(block) < 11:
#             block += '0' * (11 - len(block))
#         block_vector = np.array(list(map(int, block)))
#         encoded_vector = np.dot(block_vector, pgm) % 2
#         encoded_blocks.append(''.join(map(str, encoded_vector)))
    
#     return ''.join(encoded_blocks)

# def permute_rows():
#     global pgm
#     pgm[[0, 1]] = pgm[[1, 0]]
#     update_pgm_display()

# def restore_pgm():
#     global pgm
#     pgm = np.copy(pgm_original)
#     update_pgm_display()

# def update_pgm_display():
#     pgm_display.delete("1.0", tk.END)
#     pgm_str = '\n'.join(' '.join(str(cell) for cell in row) for row in pgm)
#     pgm_display.insert("1.0", pgm_str)

# # Set up the GUI
# root = tk.Tk()
# root.title("Hamming Encoder GUI")

# text_input = tk.Text(root, height=5, width=50)
# text_input.pack()

# load_button = tk.Button(root, text="Load Text File", command=load_text)
# load_button.pack()

# encode_button = tk.Button(root, text="Encode", command=encode)
# encode_button.pack()

# permute_button = tk.Button(root, text="Permute Rows", command=permute_rows)
# permute_button.pack()

# restore_button = tk.Button(root, text="Restore PGM", command=restore_pgm)
# restore_button.pack()

# pgm_display = tk.Text(root, height=11, width=75, bg="dark grey")
# pgm_display.pack()
# update_pgm_display()

# encoded_output = tk.Text(root, height=5, width=50, bg="dark cyan")
# encoded_output.pack()

# root.mainloop()